# Kaggle Playground - Season 4 Episode 
## Binary Classification of Insurance Cross Selling

Competion link - https://www.kaggle.com/competitions/playground-series-s4e8

### Steps
- Import the necessary libraries, packages and modules
- Read the datsets as data framers

### Understand the problem

- class is the target variable
- It determines the class of a mushroom depending on the given variables

In [54]:
# Import the necessary libraries, packages and modules

import warnings
warnings.filterwarnings('ignore')

# import dtale    # Use of a web progrm to analysis the data deeply
import lightgbm as lgb
import logging
import matplotlib.pyplot as plt
import numpy as np
import optuna
import os
import pandas as pd
import pickle
import seaborn as sns
# import statsmodels.api as sm
import tensorflow as tf
import time
import xgboost as xgb
# import zipfile

from imblearn.over_sampling import RandomOverSampler
from optuna.samplers import TPESampler
#from pandas_profiling import ProfileReport
from scipy.stats import randint, uniform
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, HistGradientBoostingClassifier)
from sklearn.feature_selection import chi2, RFE, SelectKBest, SelectFromModel  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# from skopt import BayesSearchCV
from xgboost import XGBClassifier

sns.set()
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
start_time = time.time()

In [2]:
# Test to see if TensorFlow can utilize the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
train_df = pd.read_csv('train.csv', index_col = 'id')
test_df = pd.read_csv('test.csv', index_col = 'id')

logger.info(f"Train data load completed. Time elapsed: {time.time() - start_time:.2f} seconds")

train_df.head(2)

INFO:__main__:Train data load completed. Time elapsed: 7.52 seconds


class  cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
id                                                                            
0      e          8.80         f           s         u                    f   
1      p          4.51         x           h         o                    f   

   gill-attachment gill-spacing gill-color  stem-height  ...  stem-root  \
id                                                       ...              
0                a            c          w         4.51  ...        NaN   
1                a            c          n         4.79  ...        NaN   

   stem-surface stem-color veil-type veil-color has-ring ring-type  \
id                                                                   
0           NaN          w       NaN        NaN        f         f   
1             y          o       NaN        NaN        t         z   

   spore-print-color habitat season  
id                                   
0                NaN       d      a  
1                NaN       d      w  

[2 rows x 21 columns]

In [4]:
test_df.head(2)

cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
id                                                                           
3116945          8.64         x         NaN         n                    t   
3116946          6.90         o           t         o                    f   

        gill-attachment gill-spacing gill-color  stem-height  stem-width  \
id                                                                         
3116945             NaN          NaN          w        11.13       17.12   
3116946             NaN            c          y         1.27       10.75   

        stem-root stem-surface stem-color veil-type veil-color has-ring  \
id                                                                        
3116945         b          NaN          w         u          w        t   
3116946       NaN          NaN          n       NaN        NaN        f   

        ring-type spore-print-color habitat season  
id                                                  
3116945         g               NaN       d      a  
3116946         f               NaN       d      a

### Identify the target variable and features

- class is the target variable
- It determines the class of a mushroom depending on the given variables

### Remove duplicate rows

- Checked the sum of duplicated rows in train and test datasets
- No dupllicated rows in train dataframe

### Handling missing values

- Checked the missing values in column
- There are considerable amount of missing values in many columns

In [5]:
train_df.shape

(3116945, 21)

In [6]:
test_df.shape

(2077964, 20)

### Data Preprocessing
- Droping columns with more than 50% missing values
- Using simple imputer
- Encoding Categorical Variables

In [7]:
# Drop columns with extremely high missing values
columns_to_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color', 'stem-surface']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

In [8]:
# Spliting dependent and independent valriable

y_train = train_df['class']
train_df = train_df.drop('class', axis = 1)

train_df.head(2)

cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
id                                                                      
0           8.80         f           s         u                    f   
1           4.51         x           h         o                    f   

   gill-attachment gill-spacing gill-color  stem-height  stem-width  \
id                                                                    
0                a            c          w         4.51       15.39   
1                a            c          n         4.79        6.48   

   stem-color has-ring ring-type habitat season  
id                                               
0           w        f         f       d      a  
1           o        t         z       d      w

In [9]:
# Identify numerical and categorical columns
numerical_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train_df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print("Unique Value is each categorical column :")
    print(col, train_df[col].nunique())

Unique Value is each categorical column :
cap-shape 74
Unique Value is each categorical column :
cap-surface 83
Unique Value is each categorical column :
cap-color 78
Unique Value is each categorical column :
does-bruise-or-bleed 26
Unique Value is each categorical column :
gill-attachment 78
Unique Value is each categorical column :
gill-spacing 48
Unique Value is each categorical column :
gill-color 63
Unique Value is each categorical column :
stem-color 59
Unique Value is each categorical column :
has-ring 23
Unique Value is each categorical column :
ring-type 40
Unique Value is each categorical column :
habitat 52
Unique Value is each categorical column :
season 4


In [10]:
le = LabelEncoder()

# Define the imputer for numerical columns (iterative imputation)
numerical_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(random_state = 42)),
    ('scaler', StandardScaler())
])

# Define a function to apply LabelEncoder to each categorical column
def encode_labels(df, columns):
    df_encoded = df.copy()
    le_dict = {}
    for col in columns:
        # le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        le_dict[col] = le
    return df_encoded, le_dict

# Define the imputer and encoder for categorical columns using LabelEncoder
def encode_labels_pipeline(X):
    df = pd.DataFrame(X, columns=categorical_cols)
    df_encoded, le_dict = encode_labels(df, categorical_cols)
    return df_encoded.values

# Create a ColumnTransformer for numerical and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', FunctionTransformer(func=lambda X: encode_labels_pipeline(X), validate=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Fit the preprocessor on the training data
preprocessor.fit(train_df)

# Transform both the training and test data
train_df_transformed = preprocessor.transform(train_df)
test_df_transformed = preprocessor.transform(test_df)

print("Transformed Training Data Shape:", train_df_transformed.shape)
print("Transformed Test Data Shape:", test_df_transformed.shape)

logger.info(f"Missing values and categorical columns treatment completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Missing values and categorical columns treatment completed. Time elapsed: 35.10 seconds


Transformed Training Data Shape: (3116945, 15)
Transformed Test Data Shape: (2077964, 15)


In [11]:
y_train = le.fit_transform(y_train)

y_train

array([0, 1, 0, ..., 1, 0, 1])

In [12]:
# Combine feature names
all_feature_names = numerical_cols.tolist() + categorical_cols.tolist()

# Convert to DataFrame
train_df_transformed = pd.DataFrame(train_df_transformed, columns=all_feature_names)
test_df_transformed = pd.DataFrame(test_df_transformed, columns=all_feature_names)

In [13]:
def optimize_memory_usage(df):
    start_mem_usage = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type.name in ['category', 'object']:
            raise ValueError(f"Column '{col}' is of type '{col_type.name}'")

        c_min = df[col].min()
        c_max = df[col].max()
        
        if str(col_type)[:3] == 'int':
            
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
                
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
                
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
                
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
        
        else:
        
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            
            else:
                df[col] = df[col].astype(np.float64)

    end_mem_usage = df.memory_usage().sum() / 1024**2
    print(f'------ Memory usage before: {start_mem_usage:.2f} MB')
    print(f'------ Memory usage after: {end_mem_usage:.2f} MB')
    print(f'------ Reduced memory usage by {(100 * (start_mem_usage - end_mem_usage) / start_mem_usage):.1f}%')
    print('**********************' * 5)

    return df

In [14]:
train_df_transformed = optimize_memory_usage(train_df_transformed)
test_df_transformed = optimize_memory_usage(test_df_transformed)

------ Memory usage before: 356.71 MB
------ Memory usage after: 89.18 MB
------ Reduced memory usage by 75.0%
**************************************************************************************************************
------ Memory usage before: 237.80 MB
------ Memory usage after: 59.45 MB
------ Reduced memory usage by 75.0%
**************************************************************************************************************


In [15]:
train_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.534668    -0.681152    0.523438       53.0         72.0       72.0   
1     -0.386475    -0.577148   -0.577148       71.0         56.0       64.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                   8.0             44.0          28.0        59.0   
1                   8.0             44.0          28.0        46.0   

   stem-color  has-ring  ring-type  habitat  season  
0        55.0       5.0       18.0     25.0     0.0  
1        47.0      18.0       39.0     25.0     3.0

In [16]:
test_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.500488     1.771484    0.736816       59.0         59.0       44.0   
1      0.126709    -1.880859   -0.049866       50.0         53.0       45.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                  18.0             66.0          35.0        52.0   
1                   5.0             66.0          17.0        54.0   

   stem-color  has-ring  ring-type  habitat  season  
0        51.0      17.0       15.0     16.0     0.0  
1        38.0       6.0       14.0     16.0     0.0

In [17]:
train_df_transformed['class'] = y_train
train_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.534668    -0.681152    0.523438       53.0         72.0       72.0   
1     -0.386475    -0.577148   -0.577148       71.0         56.0       64.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                   8.0             44.0          28.0        59.0   
1                   8.0             44.0          28.0        46.0   

   stem-color  has-ring  ring-type  habitat  season  class  
0        55.0       5.0       18.0     25.0     0.0      0  
1        47.0      18.0       39.0     25.0     3.0      1

In [18]:
# Since we have only one data set, spliting it into train and test (validation)

train_df_final, validation_df = train_test_split(train_df_transformed, train_size = 0.75, random_state = 42, stratify = train_df_transformed['class'])
logger.info(f"Train test split completed. Time elapsed: {time.time() - start_time:.2f} seconds")
train_df_final.head(2)

INFO:__main__:Train test split completed. Time elapsed: 38.49 seconds


cap-diameter  stem-height  stem-width  cap-shape  cap-surface  \
2419543      0.169678    -0.025314    0.197144       67.0         72.0   
910367       0.171753     0.108032   -0.094360       53.0         83.0   

         cap-color  does-bruise-or-bleed  gill-attachment  gill-spacing  \
2419543       64.0                   8.0             47.0          31.0   
910367        76.0                   8.0             75.0          28.0   

         gill-color  stem-color  has-ring  ring-type  habitat  season  class  
2419543        27.0        55.0       5.0       18.0     29.0     3.0      0  
910367         46.0        55.0       5.0       18.0     25.0     0.0      1

In [19]:
# Feature Selection
def feature_selection(X_train, y_train, model):
    if hasattr(model, 'coef_') or hasattr(model, 'feature_importances_'):
        selector = SelectFromModel(model, threshold='mean')
        selector.fit(X_train, y_train)
        return selector.transform(X_train), selector.get_support()
    else:
        raise ValueError("Feature selection not supported for this model.")

def alternative_feature_selection(X_train, y_train):
    selector = SelectKBest(score_func=f_classif, k='all')
    X_train_selected = selector.fit_transform(X_train, y_train)
    return X_train_selected, selector.get_support()

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'Ridge Classifier': RidgeClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Bagging Classifier': BaggingClassifier(),
    'Random Forest': RandomForestClassifier(),
    'AdaBoost Classifier': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'SVC': SVC(),
#    'KNN': KNeighborsClassifier(),
    'XGBoost': xgb.XGBClassifier(tree_method='gpu_hist'),
    'LightGBM': lgb.LGBMClassifier(device='gpu')
}

# Define parameter grids for RandomizedSearchCV
param_grids = {
    'Logistic Regression': {
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
        'penalty': ['l2', 'none'],  # 'elasticnet' is not supported by 'lbfgs'
        'C': uniform(0.001, 1000),  # Inverse of regularization strength
        'max_iter': [100, 200, 300]
    },
    'Ridge Classifier': {
        'alpha': uniform(0.1, 10),  # Regularization strength
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg'],
        'fit_intercept': [True, False],
        'max_iter': [100, 200, 300],
       # 'normalize': [True, False]  # This should be removed as it's not a valid parameter
    },
    'Decision Tree': {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': randint(3, 20),
    'min_samples_split': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_samples_leaf': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_weight_fraction_leaf': uniform(0.0, 0.1),  # Random float between 0.0 and 0.1
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_leaf_nodes': randint(10, 50),
    'min_impurity_decrease': uniform(0.0, 0.1),
    'class_weight': [None, 'balanced']
},
    'Bagging Classifier': {
    'base_estimator': [DecisionTreeClassifier(), None],  # Default is DecisionTreeClassifier
    'n_estimators': randint(10, 100),
    'max_samples': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'max_features': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42]  # Set to None for random results, or a fixed integer for reproducibility
},
    'Random Forest': {
    'n_estimators': randint(50, 300),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, randint(3, 20)],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', uniform(0.5, 0.5)],
    'bootstrap': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42],
    'verbose': [0, 1],
    'warm_start': [True, False],
    'class_weight': [None, 'balanced']
},
    'AdaBoost Classifier': {
    'base_estimator': [None, DecisionTreeClassifier(max_depth=1)],
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 1.0),
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [42]
},
    'Gradient Boosting': {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.5),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None, uniform(0.1, 0.9)],
    'subsample': uniform(0.5, 0.5),
    'criterion': ['friedman_mse', 'squared_error', 'mae'],
    #'alpha': uniform(0.5, 0.5),
    'random_state': [42],
    'verbose': [0, 1]
},
    'SVC': {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': randint(2, 5),
    'gamma': ['scale', 'auto', uniform(0.001, 1)],
    'coef0': uniform(0, 10),
    'shrinking': [True, False],
    'probability': [True, False],
    'tol': uniform(1e-5, 1e-1),
    'cache_size': uniform(50, 500),
    'class_weight': [None, 'balanced'],
    'verbose': [True, False],
    'max_iter': [-1, 100, 200],
    'decision_function_shape': ['ovr', 'ovo'],
    'break_ties': [True, False],
    'random_state': [42]
},
    'KNN': {
    'n_neighbors': randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': randint(1, 3),
    'metric': ['minkowski', 'euclidean', 'manhattan', 'chebyshev', 'hamming'],
    'leaf_size': randint(10, 50),
    'n_jobs': [None, -1]
},
    'XGBoost': {
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 0.5),
    'min_child_weight': randint(1, 10),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10),
    'max_delta_step': randint(0, 10),
    'colsample_bylevel': uniform(0.5, 0.5),
    'colsample_bynode': uniform(0.5, 0.5)
},
    'LightGBM': {
    'num_leaves': randint(20, 100),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 300),
    'min_child_samples': randint(10, 100),
    'min_split_gain': uniform(0, 0.5),
    'subsample': uniform(0.5, 0.5),
    'subsample_freq': randint(1, 10),
    'colsample_bytree': uniform(0.5, 0.5),
    'colsample_bylevel': uniform(0.5, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10)
}
}

# Optuna objective function for XGBoost and LightGBM
def objective(trial, model_name, model, X_train, y_train, X_val, y_val):
    if model_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_uniform('gamma', 0, 0.5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 1),
            'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1, 10),
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
            'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
            'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.5, 1.0)
        }
    elif model_name == 'LightGBM':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
            'num_leaves': trial.suggest_int('num_leaves', 31, 127),
            'max_depth': trial.suggest_int('max_depth', -1, 15),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 0.2),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 0.2),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
            'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 0.5),
            'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
            'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
            'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1, 10)
        }

    model.set_params(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = matthews_corrcoef(y_val, preds)
    return score

# Hyperparameter tuning using RandomizedSearchCV
def tune_model(model_name, model, param_grid, X_train, y_train):
    search = RandomizedSearchCV(
        model, param_distributions=param_grid, n_iter=10, scoring='f1', cv=3, random_state=42
    )
    search.fit(X_train, y_train)
    print(f"{model_name} Best Parameters (Random Search): {search.best_params_}")  # Print best parameters
    return search.best_estimator_, search.best_params_

# Hyperparameter tuning using Optuna for XGBoost and LightGBM
def optuna_tune_model(model_name, model, X_train, y_train, X_val, y_val):
    study = optuna.create_study(direction='maximize', sampler=TPESampler())
    study.optimize(lambda trial: objective(trial, model_name, model, X_train, y_train, X_val, y_val), n_trials=50)
    print(f"{model_name} Best Parameters (Optuna): {study.best_params}")  # Print best parameters
    return study.best_params

# Train and evaluate models
def evaluate_models(X_train, y_train, X_val, y_val):
    best_models = {}
    for name, model in models.items():
        logger.info(f"Evaluating {name}")

        # Feature Selection
        if hasattr(model, 'coef_') or hasattr(model, 'feature_importances_'):
            logger.info(f"Performing feature selection for {name}")
            X_train_selected, support = feature_selection(X_train, y_train, model)
            X_val_selected = X_val.iloc[:, support]
        else:
            logger.info(f"Skipping feature selection for {name} due to lack of support")
            X_train_selected = X_train
            X_val_selected = X_val

        if name in ['XGBoost', 'LightGBM']:
            # Perform Random Search
            best_model, best_params_random = tune_model(name, model, param_grids[name], X_train_selected, y_train)
            train_preds_random = best_model.predict(X_train_selected)
            val_preds_random = best_model.predict(X_val_selected)
            train_score_random = matthews_corrcoef(y_train, train_preds_random)
            val_score_random = matthews_corrcoef(y_val, val_preds_random)
            logger.info(f"{name} (Random Search) - Train MCC Score: {train_score_random:.4f}")
            logger.info(f"{name} (Random Search) - Validation MCC Score: {val_score_random:.4f}")
            
            # Perform Bayesian Search (Optuna)
            params_optuna = optuna_tune_model(name, model, X_train_selected, y_train, X_val_selected, y_val)
            model.set_params(**params_optuna)
            model.fit(X_train_selected, y_train)
            train_preds_optuna = model.predict(X_train_selected)
            val_preds_optuna = model.predict(X_val_selected)
            train_score_optuna = matthews_corrcoef(y_train, train_preds_optuna)
            val_score_optuna = matthews_corrcoef(y_val, val_preds_optuna)
            logger.info(f"{name} (Optuna) - Train MCC Score: {train_score_optuna:.4f}")
            logger.info(f"{name} (Optuna) - Validation MCC Score: {val_score_optuna:.4f}")
            logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")
            
        else:
            # Hyperparameter tuning using RandomizedSearchCV
            best_model, best_params = tune_model(name, model, param_grids[name], X_train_selected, y_train)
            model = best_model
            train_preds = model.predict(X_train_selected)
            val_preds = model.predict(X_val_selected)
            train_score = matthews_corrcoef(y_train, train_preds)
            val_score = matthews_corrcoef(y_val, val_preds)
            logger.info(f"{name} - Train MCC Score: {train_score:.4f}")
            logger.info(f"{name} - Validation MCC Score: {val_score:.4f}")
            logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

        best_models[name] = model

    return best_models

# Testing best models on the test set
def test_best_models(best_models, X_train, y_train, X_val, y_val):
    results = {}
    for name, model in best_models.items():
        # Training predictions and score
        train_preds = model.predict(X_train)
        train_score = matthews_corrcoef(y_train, train_preds)

        # Validation predictions and score
        val_preds = model.predict(X_val)
        val_score = matthews_corrcoef(y_val, val_preds)

        # Logging the scores
        logger.info(f"{name} - Train MCC Score: {train_score:.4f}")
        logger.info(f"{name} - Validation MCC Score: {val_score:.4f}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

        # Storing the results
        results[name] = {
            'train_score': train_score,
            'val_score': val_score,
            #'test_score': test_score
        }
    return results

In [20]:
y_train_final = train_df_final['class']
train_df_final = train_df_final.drop('class', axis = 1)

y_val = validation_df['class']
validation_df = validation_df.drop('class', axis = 1)

In [21]:
# Evaluate models
best_models = evaluate_models(train_df_final, y_train_final, validation_df, y_val)

logger.info(f"Model evaluation completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Evaluating Logistic Regression
INFO:__main__:Skipping feature selection for Logistic Regression due to lack of support


Logistic Regression Best Parameters (Random Search): {'C': 181.82596720710063, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}


INFO:__main__:Logistic Regression - Train MCC Score: 0.2324
INFO:__main__:Logistic Regression - Validation MCC Score: 0.2336
INFO:__main__:Time elapsed: 1417.98 seconds
INFO:__main__:Evaluating Ridge Classifier
INFO:__main__:Skipping feature selection for Ridge Classifier due to lack of support


Ridge Classifier Best Parameters (Random Search): {'alpha': 5.347746602583891, 'fit_intercept': True, 'max_iter': 200, 'solver': 'lsqr'}


INFO:__main__:Ridge Classifier - Train MCC Score: 0.2340
INFO:__main__:Ridge Classifier - Validation MCC Score: 0.2350
INFO:__main__:Time elapsed: 1472.74 seconds
INFO:__main__:Evaluating Decision Tree
INFO:__main__:Skipping feature selection for Decision Tree due to lack of support


...

[I 2024-08-14 12:18:42,172] Trial 48 finished with value: 0.9821189695858932 and parameters: {'n_estimators': 168, 'learning_rate': 0.1813484609179948, 'num_leaves': 101, 'max_depth': 13, 'subsample': 0.5563882395293387, 'colsample_bytree': 0.5356772755023653, 'reg_alpha': 0.17195976527084803, 'reg_lambda': 0.17384962137486282, 'min_child_samples': 42, 'min_split_gain': 0.14786525092277153, 'subsample_freq': 7, 'colsample_bylevel': 0.7519800424107033, 'scale_pos_weight': 1.0043387082989392}. Best is trial 44 with value: 0.982258744587561.


[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 927
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce MX250, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (26.75 MB) transferred to GPU in 0.071621 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.78 MB) transferred to GPU in 0.064357 secs. 1 sparse fea

[I 2024-08-14 12:19:37,611] Trial 49 finished with value: 0.9739722579778499 and parameters: {'n_estimators': 167, 'learning_rate': 0.10329681430912153, 'num_leaves': 101, 'max_depth': 15, 'subsample': 0.8513231445686431, 'colsample_bytree': 0.5252304484310223, 'reg_alpha': 0.15231254569265734, 'reg_lambda': 0.1496449307515921, 'min_child_samples': 41, 'min_split_gain': 0.15177725368709827, 'subsample_freq': 7, 'colsample_bylevel': 0.7563596205319121, 'scale_pos_weight': 6.286385450626617}. Best is trial 44 with value: 0.982258744587561.


LightGBM Best Parameters (Optuna): {'n_estimators': 171, 'learning_rate': 0.1629222720994061, 'num_leaves': 107, 'max_depth': 10, 'subsample': 0.5253943247464046, 'colsample_bytree': 0.5521100245389093, 'reg_alpha': 0.18740739028052475, 'reg_lambda': 0.1902519135478532, 'min_child_samples': 50, 'min_split_gain': 0.1354971299775326, 'subsample_freq': 5, 'colsample_bylevel': 0.7683520039190338, 'scale_pos_weight': 1.0329477234594147}
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 927
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce MX250, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM]

INFO:__main__:LightGBM (Optuna) - Train MCC Score: 0.9831
INFO:__main__:LightGBM (Optuna) - Validation MCC Score: 0.9823
INFO:__main__:Time elapsed: 49151.05 seconds
INFO:__main__:Model evaluation completed. Time elapsed: 49151.06 seconds


In [22]:
best_models

{'Logistic Regression': LogisticRegression(C=181.82596720710063, solver='saga'),
 'Ridge Classifier': RidgeClassifier(alpha=5.347746602583891, max_iter=200, solver='lsqr'),
 'Decision Tree': DecisionTreeClassifier(criterion='entropy', max_depth=17, max_features='log2',
                        max_leaf_nodes=17,
                        min_impurity_decrease=0.05986584841970366,
                        min_samples_leaf=0.025601864044243652,
                        min_samples_split=0.025599452033620268,
                        min_weight_fraction_leaf=0.005808361216819946,
                        splitter='random'),
 'Bagging Classifier': BaggingClassifier(base_estimator=None, bootstrap=False,
                   max_features=0.6039708314340944,
                   max_samples=0.7838501639099957, n_estimators=72, n_jobs=-1,
                   random_state=42),
 'Random Forest': RandomForestClassifier(class_weight='balanced', max_features='log2',
                        min_samples_leaf=8, 

In [32]:
results = test_best_models(best_models, train_df_final, y_train_final, validation_df, y_val)

INFO:__main__:Logistic Regression - Train MCC Score: 0.2324
INFO:__main__:Logistic Regression - Validation MCC Score: 0.2336
INFO:__main__:Time elapsed: 64458.45 seconds
INFO:__main__:Ridge Classifier - Train MCC Score: 0.2340
INFO:__main__:Ridge Classifier - Validation MCC Score: 0.2350
INFO:__main__:Time elapsed: 64459.96 seconds
INFO:__main__:Decision Tree - Train MCC Score: 0.0000
INFO:__main__:Decision Tree - Validation MCC Score: 0.0000
INFO:__main__:Time elapsed: 64461.35 seconds
INFO:__main__:Bagging Classifier - Train MCC Score: 0.9990
INFO:__main__:Bagging Classifier - Validation MCC Score: 0.9828
INFO:__main__:Time elapsed: 64491.16 seconds
INFO:__main__:Random Forest - Train MCC Score: 0.9837
INFO:__main__:Random Forest - Validation MCC Score: 0.9820
INFO:__main__:Time elapsed: 64630.64 seconds
INFO:__main__:AdaBoost Classifier - Train MCC Score: 0.6550
INFO:__main__:AdaBoost Classifier - Validation MCC Score: 0.6553
INFO:__main__:Time elapsed: 64766.46 seconds
INFO:__main_

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: colsample_bylevel


INFO:__main__:LightGBM - Train MCC Score: 0.9831
INFO:__main__:LightGBM - Validation MCC Score: 0.9823
INFO:__main__:Time elapsed: 64876.52 seconds


In [24]:
test_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.500488     1.771484    0.736816       59.0         59.0       44.0   
1      0.126709    -1.880859   -0.049866       50.0         53.0       45.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                  18.0             66.0          35.0        52.0   
1                   5.0             66.0          17.0        54.0   

   stem-color  has-ring  ring-type  habitat  season  
0        51.0      17.0       15.0     16.0     0.0  
1        38.0       6.0       14.0     16.0     0.0

In [34]:
# Initialize logging
logging.info("Starting cross-validation process")
logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

# Sort the models by their validation scores in descending order
sorted_test_models = sorted(results.items(), key=lambda item: item[1]['val_score'], reverse=True)

# Select the top 5 models based on validation scores
five_best_models = {model_name: scores for model_name, scores in sorted_test_models[:5]}

# Output the best five models based on validation scores
for model_name, scores in five_best_models.items():
    print(f"Model: {model_name}, Validation MCC Score: {scores['val_score']}, Train MCC Score: {scores['train_score']}")

# The best_five_test_models dictionary now contains the top five models based on validation data performance


INFO:root:Starting cross-validation process
INFO:__main__:Time elapsed: 65311.59 seconds


Model: Bagging Classifier, Validation MCC Score: 0.9828474395791728, Train MCC Score: 0.9990228627484805
Model: LightGBM, Validation MCC Score: 0.9822763182496796, Train MCC Score: 0.9831339119250782
Model: XGBoost, Validation MCC Score: 0.9822273508286966, Train MCC Score: 0.9830919851466178
Model: Random Forest, Validation MCC Score: 0.9819531641457832, Train MCC Score: 0.9836635807660468
Model: Gradient Boosting, Validation MCC Score: 0.9770009203293313, Train MCC Score: 0.9773071424994347


In [35]:
train_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.534668    -0.681152    0.523438       53.0         72.0       72.0   
1     -0.386475    -0.577148   -0.577148       71.0         56.0       64.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                   8.0             44.0          28.0        59.0   
1                   8.0             44.0          28.0        46.0   

   stem-color  has-ring  ring-type  habitat  season  class  
0        55.0       5.0       18.0     25.0     0.0      0  
1        47.0      18.0       39.0     25.0     3.0      1

In [36]:
y_train

array([0, 1, 0, ..., 1, 0, 1])

In [46]:
test_df_sub = pd.read_csv('test.csv')
test_df_sub.head(2)

id  cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
0  3116945          8.64         x         NaN         n                    t   
1  3116946          6.90         o           t         o                    f   

  gill-attachment gill-spacing gill-color  stem-height  ...  stem-root  \
0             NaN          NaN          w        11.13  ...          b   
1             NaN            c          y         1.27  ...        NaN   

  stem-surface stem-color veil-type veil-color has-ring ring-type  \
0          NaN          w         u          w        t         g   
1          NaN          n       NaN        NaN        f         f   

  spore-print-color habitat season  
0               NaN       d      a  
1               NaN       d      a  

[2 rows x 21 columns]

In [39]:
y_train[1].dtype

dtype('int32')

In [40]:
train_df_transformed.drop('class', axis=1, inplace=True)
train_df_transformed.head(2)

cap-diameter  stem-height  stem-width  cap-shape  cap-surface  cap-color  \
0      0.534668    -0.681152    0.523438       53.0         72.0       72.0   
1     -0.386475    -0.577148   -0.577148       71.0         56.0       64.0   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                   8.0             44.0          28.0        59.0   
1                   8.0             44.0          28.0        46.0   

   stem-color  has-ring  ring-type  habitat  season  
0        55.0       5.0       18.0     25.0     0.0  
1        47.0      18.0       39.0     25.0     3.0

In [51]:
# Define the Train_ML function
def Train_ML(Model, X, y, test_data):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    train_scores = []
    val_scores = []
    test_predictions = []

    for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
        # Handle indexing based on the type of X and y
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        else:
            X_train, X_val = X[train_index], X[val_index]
        
        if isinstance(y, pd.Series):
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        else:
            y_train, y_val = y[train_index], y[val_index]
        
        Model.fit(X_train, y_train)

        y_train_pred = Model.predict(X_train)
        train_mcc = matthews_corrcoef(y_train, y_train_pred)
        train_scores.append(train_mcc)

        y_val_pred = Model.predict(X_val)
        val_mcc = matthews_corrcoef(y_val, y_val_pred)
        val_scores.append(val_mcc)
        
        y_test_pred_proba = Model.predict(test_data)
        test_predictions.append(y_test_pred_proba)

        print(f"Fold {fold}: Train MCC = {train_mcc:.6f}, Validation MCC = {val_mcc:.6f}")
        logger.info(f"Fold {fold}: Train MCC = {train_mcc:.6f}, Validation MCC = {val_mcc:.6f}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

    mean_train_mcc = np.mean(train_scores)
    mean_val_mcc = np.mean(val_scores)

    print(f"\nMean Train MCC: {mean_train_mcc:.6f}")
    print(f"Mean Validation MCC: {mean_val_mcc:.6f}")
    logger.info(f"Mean Train MCC: {mean_train_mcc:.6f}, Mean Validation MCC: {mean_val_mcc:.6f}")
    logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

    return Model, test_predictions

In [49]:
five_best_models

{'Bagging Classifier': {'train_score': 0.9990228627484805,
  'val_score': 0.9828474395791728},
 'LightGBM': {'train_score': 0.9831339119250782,
  'val_score': 0.9822763182496796},
 'XGBoost': {'train_score': 0.9830919851466178,
  'val_score': 0.9822273508286966},
 'Random Forest': {'train_score': 0.9836635807660468,
  'val_score': 0.9819531641457832},
 'Gradient Boosting': {'train_score': 0.9773071424994347,
  'val_score': 0.9770009203293313}}

In [56]:
# Perform cross-validation, fit on the entire training data, and predict for each model
for model_name, model in five_best_models.items():
    try:
        logger.info(f"Starting model training and cross-validation for {model_name}")
        trained_model, test_preds = Train_ML(best_models[model_name], train_df_transformed, y_train, test_df_transformed)

        # Averaging predictions across all folds
        final_test_preds = np.mean(test_preds, axis=0)
        final_test_preds_binary = (final_test_preds >= 0.5).astype(int)
        
        # Inverse transform the predictions to get the original class labels
        predictions = le.inverse_transform(final_test_preds_binary)

        # Log the prediction output
        logger.info(f"{model_name} - Test predictions done")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

        # Output predictions to a CSV file
        output_df = pd.DataFrame({'id': test_df_sub['id'], 'class': predictions})
        output_df.to_csv(f'Submission_02A_Dropped_Iterative_{model_name}.csv', index=False)
        print(output_df.head(2))
        logger.info(f"Generated output file for {model_name}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

    except Exception as e:
        logger.error(f"An error occurred with {model_name}: {e}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

# Print completion message
print("Predictions for all five models have been saved to individual CSV files.")
logger.info("Predictions for all five models have been saved to individual CSV files.")
logger.info(f"Total Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Starting model training and cross-validation for Bagging Classifier
INFO:__main__:Fold 1: Train MCC = 0.998967, Validation MCC = 0.982903
INFO:__main__:Time elapsed: 6548.87 seconds


Fold 1: Train MCC = 0.998967, Validation MCC = 0.982903


INFO:__main__:Fold 2: Train MCC = 0.998961, Validation MCC = 0.982973
INFO:__main__:Time elapsed: 7114.22 seconds


Fold 2: Train MCC = 0.998961, Validation MCC = 0.982973


INFO:__main__:Fold 3: Train MCC = 0.998976, Validation MCC = 0.982575
INFO:__main__:Time elapsed: 7688.31 seconds


...

Fold 4: Train MCC = 0.977509, Validation MCC = 0.976957


INFO:__main__:Fold 5: Train MCC = 0.977447, Validation MCC = 0.977547
INFO:__main__:Time elapsed: 12633.03 seconds
INFO:__main__:Mean Train MCC: 0.977353, Mean Validation MCC: 0.977065
INFO:__main__:Time elapsed: 12633.04 seconds
INFO:__main__:Gradient Boosting - Test predictions done
INFO:__main__:Time elapsed: 12633.16 seconds


Fold 5: Train MCC = 0.977447, Validation MCC = 0.977547

Mean Train MCC: 0.977353
Mean Validation MCC: 0.977065


INFO:__main__:Generated output file for Gradient Boosting
INFO:__main__:Time elapsed: 12635.70 seconds
INFO:__main__:Predictions for all five models have been saved to individual CSV files.
INFO:__main__:Total Time elapsed: 12635.70 seconds


Predictions for all five models have been saved to individual CSV files.


In [59]:
# Define MCC as the scoring metric
mcc_scorer = make_scorer(matthews_corrcoef)

# Perform cross-validation, fit on the entire training data, and predict for each model
predictions = {}
for model_name, model in five_best_models.items():
    try:
        logging.info(f"Performing cross-validation for {model_name}")
        scores = cross_val_score(best_models[model_name], train_df_transformed, y_train, cv=5, scoring=mcc_scorer, n_jobs=-1)
        logging.info(f"{model_name} - CV Scores: {scores}")
        logging.info(f"{model_name} - Mean CV Score: {np.mean(scores)}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")
    
        # Fit the model on the entire training data
        best_models[model_name].fit(train_df_transformed, y_train)
        
        # Predict the output for test_df_transformed
        preds = best_models[model_name].predict(test_df_transformed)
    
        # Inverse transform the predictions to get the original class labels
        predictions[model_name] = le.inverse_transform(preds)
    
        # Log the prediction output
        logging.info(f"{model_name} - Test predictions done for {model_name}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")
    
        # Output predictions to a CSV file
        output_df = pd.DataFrame({'id': test_df_sub['id'], 'class': predictions[model_name]})
        output_df.to_csv(f'Submission_02_Dropped_Iterative_{model_name}.csv', index=False)
        print(output_df.head(2))
        logger.info(f"Generated output file - Time elapsed: {time.time() - start_time:.2f} seconds")

    except Exception as e:
        logger.error(f"An error occurred with {model_name}: {e}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

# Print completion message
print("Predictions for all five models have been saved to individual CSV files.")
logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:root:Performing cross-validation for Bagging Classifier


KeyboardInterrupt: 